In [1]:
pip install razdel

In [2]:
import pandas as pd
from razdel import sentenize

data = pd.read_excel(r"/content/sample_data/answers_base.xlsx")
#print(data.columns)

data = data.dropna()
pd.set_option('display.max_columns', None)
#print(data)

all_ans = []
g_data = []
[all_ans.append(row[2]) for row in data.itertuples()]
for ans in all_ans:
  tokens = sentenize(ans)
  text = [_.text for _ in tokens] 
  g_data.extend(text)
#all_ans = all_ans[:200]
print(g_data)

['У ребенка в школе продлили каникулы.', 'Могу ли я взять больничный на это время?', 'Больничный лист?', 'Есть ли компенсация, в случае если есть разница между оплатой больничного и зарплатой?\nкак оплачивается больничный при коронавирусе?', 'Я контактный, дадут ли больничный?', 'Кто компенсирует разницу между оплатой больничного и реальной зарплатой, если я был в самоизоляции и здоров?', 'Как я могу получить больничный лист, если я нахожусь на самоизоляции?', 'Дадут ли больничный, контактировал с больным?', 'Не могу получить дистанционный оплачиваемый больничный, потому что страна, откуда я вернулся, не содержится в списке стран, после посещения которых необходимо соблюдать карантин?', 'Листок нетрудоспособности?', 'Больничный по контакту?', 'На какой срок больничный?', 'Как оплатят больничный за время болезни и карантина?', 'Как получить больничный для работающих граждан 65 лет и старше?', 'Где сделать вакцинацию от коронавируса?', 'Существует ли вакцина от коронавируса?\n?', 'Когда 

In [3]:
pip install pymorphy2

     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 8.0MB 6.3MB/s 


In [4]:
pip install pymorphy2_dicts_ru

In [60]:
from razdel import tokenize
import pymorphy2 as pm
from pymystem3 import Mystem
import string
import math

morph = pm.MorphAnalyzer(path = '/usr/local/lib/python3.6/dist-packages/pymorphy2_dicts_ru/data')


def preprocessing(texts):
  for text in texts:
      all_t = [] 
      #print("processing...")
      tokens = list(tokenize(text))
      text = [_.text for _ in tokens]
      text = [w.lower() for w in text]
      table = str.maketrans('', '', string.punctuation)
      text = [word for word in text if word.isalpha()]
      text = [morph.parse(word)[0].normal_form for word in text]
      all_t.append(' '.join(text))        
  
  return all_t

In [6]:
corpus = preprocessing(g_data)
print(corpus)

['у ребёнок в школа продлить каникулы', 'мочь ли я взять больничный на это время', 'больничный лист', 'есть ли компенсация в случай если есть разница между оплата больничный и зарплата как оплачиваться больничный при коронавирус', 'я контактный дать ли больничный', 'кто компенсировать разница между оплата больничный и реальный зарплата если я быть в самоизоляция и здоровый', 'как я мочь получить больничный лист если я находиться на самоизоляция', 'дать ли больничный контактировать с больной', 'не мочь получить дистанционный оплачивать больничный потому что страна откуда я вернуться не содержаться в список страна после посещение который необходимый соблюдать карантин', 'листок нетрудоспособность', 'больничный по контакт', 'на какой срок больничный', 'как оплатить больничный за время болезнь и карантин', 'как получить больничный для работать гражданин год и старший', 'где сделать вакцинация от коронавирус', 'существовать ли вакцина от коронавирус', 'когда быть доступный вакцина от ковид'

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

query = ["карантин изоляция самоизоляция"]

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(corpus)

X = X.toarray()

new_doc = vectorizer.transform(query).toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

Теперь нужно перемножить вектор на матрицу

In [8]:
new_doc = new_doc.transpose()
vector = X.dot(new_doc)

df = pd.DataFrame(vector)
df["doc"] = g_data

In [9]:
df = df.rename(columns={0: "value_doc"})
df.sort_values(by = 'value_doc', ascending=False)

,value_doc,doc
210,2,На возвращение из каких стран распространяется...
169,2,Что такое карантин (изоляция)?
239,2,Мне нужна помощь волонтера в связи с карантино...
216,2,После возвращения нужно ли будет проходить кар...
204,2,Если я нахожусь на карантине в домашних услови...
...,...,...
283,0,Работодатель не сообщает о заболевших ковид?
284,0,Заболел сотрудник?
286,0,Применение бактерицидных облучателей на рабочи...
287,0,"Если заболели сотрудники одного предприятия, р..."


Взять общую логику bm25 отсюда: https://github.com/paauw/python-bm25/blob/master/vectorizer.py

In [32]:
from math import log
def TF(word, text):
  return len([a for a in text if word in a])

features = set([item for sublist in [a.split() for a in corpus] for item in sublist])
N = len(corpus)

def idf(word):
  n = len([text for text in corpus if word in text])
  idf_value = log((N-n+0.5)/(n+0.5))
  return idf_value

In [51]:
avgdl = 0
for text in corpus:
  avgdl += len(text)

In [52]:
k = 2.0
b = 0.75

def bm25(Q, D):
  idf_sum = 0
  tf_value = 0
  bm25_score = 0
  for q in Q:
    idf_value = idf(q)
    tf_value = TF(q, D)
    bm25_score += (idf_value * (tf_value*(k+1) / (tf_value + k*(1-b + b*(len(D)/avgdl))  ) ) )
  return bm25_score
     

In [67]:
Q = preprocessing(["карантина изоляции самоизоляция"])[0].split()
print(Q)
for D in corpus:
  print(D, bm25(Q, D.split()))

['карантин', 'изоляция', 'самоизоляция']
у ребёнок в школа продлить каникулы 0.0
мочь ли я взять больничный на это время 0.0
больничный лист 0.0
есть ли компенсация в случай если есть разница между оплата больничный и зарплата как оплачиваться больничный при коронавирус 0.0
я контактный дать ли больничный 0.0
кто компенсировать разница между оплата больничный и реальный зарплата если я быть в самоизоляция и здоровый 12.453214966772627
как я мочь получить больничный лист если я находиться на самоизоляция 12.455018316947381
дать ли больничный контактировать с больной 0.0
не мочь получить дистанционный оплачивать больничный потому что страна откуда я вернуться не содержаться в список страна после посещение который необходимый соблюдать карантин 4.764662732402632
листок нетрудоспособность 0.0
больничный по контакт 0.0
на какой срок больничный 0.0
как оплатить больничный за время болезнь и карантин 4.766732824828007
как получить больничный для работать гражданин год и старший 0.0
где сделат